# Preparation

## Libs

In [1]:
!pip install Sastrawi
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 9.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 7.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.5/381.5 kB 9.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.0 MB/s eta 0:00:00:00:010:01m


In [ ]:
pip install wordcloud

In [2]:
# Import some libraries

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
seed = 0
np.random.seed(seed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'whitegrid')

import datetime as dt
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

from google.colab import drive

import nltk
nltk.download('punkt')
nltk.download('stopwords')

ModuleNotFoundError: No module named 'wordcloud'

## df Prep

In [3]:
# from folder tweets-data in this folder merge all csv files into one dataframe pandas
import os
import glob
path = 'tweets-data'
all_files = glob.glob(os.path.join(path, "*.csv"))
df_from_each_file = (pd.read_csv(f) for f in all_files)
df = pd.concat(df_from_each_file, ignore_index=True)
df


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1840540824591020337,Sun Sep 29 23:55:02 +0000 2024,1,aku suka pemilu hehe,1840540824591020337,NaN,NaN,in,NaN,1,0,0,https://x.com/yaudahlahpasrah/status/184054082...,1094605073571672065,yaudahlahpasrah
1,1840540740847583671,Sun Sep 29 23:54:42 +0000 2024,56,Pemilu di negeri sekuat Amerika Serikat bisa d...,1840540740847583671,NaN,NaN,in,NaN,1,16,15,https://x.com/ariel_heryanto/status/1840540740...,296143572,ariel_heryanto
2,1840220844053934371,Sun Sep 29 23:51:44 +0000 2024,0,@Chrisyanto21 @raymondchins Ga ada istilah bak...,1840539991367422360,NaN,Chrisyanto21,in,Indonesia Timur,0,1,0,https://x.com/Robert32Arthur/status/1840539991...,942079081134174208,Robert32Arthur
3,1840255028575252762,Sun Sep 29 23:50:36 +0000 2024,0,@ahriesonta Dongeng lagi. Kasihan mereka cari ...,1840539706255360222,NaN,ahriesonta,in,8ft di bawah tanah,0,0,0,https://x.com/Donie707/status/1840539706255360222,1573280097003397121,Donie707
4,1840301498405925348,Sun Sep 29 23:45:45 +0000 2024,0,@LambeResing Kalau demo dia lari paling kencen...,1840538488904794500,NaN,LambeResing,in,8ft di bawah tanah,0,0,0,https://x.com/Donie707/status/1840538488904794500,1573280097003397121,Donie707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,1839945074555122152,Sun Sep 29 05:10:30 +0000 2024,24,@goraici Nunggu ada penelitian yang mengasosia...,1840257823609106770,NaN,goraici,in,Jakarta,0,0,1,https://x.com/elisa_jkt/status/184025782360910...,44556194,elisa_jkt
1554,1840010394980741484,Sun Sep 29 05:09:36 +0000 2024,0,@Adiprayitno_20 Dah biarin aja Mungkin sesuai ...,1840257599528415258,NaN,Adiprayitno_20,in,Russia,0,0,0,https://x.com/SManikmoyo/status/18402575995284...,1244476057891160064,SManikmoyo
1555,1839528669749846453,Sun Sep 29 04:54:19 +0000 2024,1,@Pegulatmasa @sayamakibao @xD34DBE3F @BuYungBi...,1840253752634421486,https://pbs.twimg.com/media/GYnlCEsbMAAbG1Q.jpg,Pegulatmasa,in,NaN,0,1,0,https://x.com/FurqanIbr64677/status/1840253752...,1669742231228682241,FurqanIbr64677
1556,1840253649722757263,Sun Sep 29 04:53:55 +0000 2024,0,Pada hari Minggu tanggal 29 September 2024 Per...,1840253649722757263,https://pbs.twimg.com/media/GYnk7lnasAINU_3.jpg,NaN,in,NaN,0,0,0,https://x.com/PolsekLing62979/status/184025364...,1768901743465897984,PolsekLing62979


## Labeling

In [4]:
from transformers import pipeline

pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"

nlp = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    tokenizer=pretrained_name
)

nlp("Jangan sampai saya telpon bos saya ya!")


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/808k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/467k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


: 